In [6]:
%load_ext autoreload
%autoreload 2

from amd_bkg_matchids import *
from excel_matches_livingscores import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. load matches

In [3]:
with open('matches_rdf.json') as f:
    matches = json.load(f)
print(len(matches))

5648


In [11]:
len(set(matches.values()))

5457

# 2. Load Booking data

In [4]:
print("Loading Booking hotels")
bkgh = load_booking(matches.values())
print(len(bkgh))

Loading Booking hotels
5457


# 2.b Load Amadeus spreadsheet

In [ ]:
print("Loading Amadeus hotels")
amdh_full = pd.read_excel('./Amadeus All Properties - FEB 2017 17022017 .xlsx', header=0, index_col='PROPERTY_CODE', skiprows=1)

In [7]:
amdh = convert_amd_df_to_matching_format(amdh_full)
amdh.amd_id = amdh.amd_id.values.astype(str)
def pc_to_bkg(pc):
    return matches.get(pc)
amdh['bkg_id'] = amdh['amd_id'].apply(pc_to_bkg)

namdh = len(amdh)
print("Loaded %d hotels" % namdh)

/mnt/data/projects/.virtualenvs/venv2/local/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Loaded 140709 hotels


# 3. Explore matches

In [10]:
amdh[amdh.bkg_id.notnull()].head()

,amd_id,lat,lng,name,chain,bkg_id
PROPERTY_CODE,,,,,,


In [8]:
matches_df = amdh.join(bkgh, on='bkg_id', how='inner', rsuffix='_bkg')
matches_df = matches_df[['amd_id', 'bkg_id_bkg',u'name', 'name_bkg', u'chain', 'chain_bkg', 'lat',  u'lng', u'lat_bkg', u'lng_bkg']]
mdf = matches_df
mdf.chain_bkg.fillna('', inplace=True)
len(matches_df)

0

In [ ]:
from collections import defaultdict
good_matches = defaultdict(list)

In [10]:
mdf[mdf.chain_bkg != ''][['chain', 'chain_bkg']].sample(50)

,chain,chain_bkg
PROPERTY_CODE,,
BTHESOSH,BUSINESS TVL ADV.,Oxford Hotel Group
XVATLBFS,SPRINGHILL SUITES,SpringHill Suites
GWBLLDAL,GREAT HOTELS,Small Danish Hotels
FNSNAVOF,FAIRFIELD INN HOTELS,Fairfield Inn
QILWS027,QUALITY INNS,Quality Inn
RTAMSEUS,ACCOR HOTELS,Mercure
DTOLM101,DOUBLETREE HOTELS,"Doubletree by Hilton, Aimbridge Hospitality"
HIDFW97C,HOLIDAY INN,Holiday Inn Express
RAYOD856,RAMADA,Ramada


# 4. clean up erroneous chain matches

In [7]:
chain_matches = load_commented_json('chain_matches_excel.json')

In [94]:
sc = mdf.copy() # candidates to suspicious

sc.name = sc.name.apply(normalize)
sc.name_bkg = sc.name_bkg.apply(normalize)
sc.chain = sc.chain.astype(unicode).apply(normalize)
sc.chain_bkg = sc.chain_bkg.astype(unicode).apply(normalize)

filter_neq_name = sc.name != sc.name_bkg
sc = sc[filter_neq_name]

len(sc)

47652

In [57]:
suspicious = pd.DataFrame(data=[],columns=mdf.columns)

for chain_a, d in chain_matches.items():
    for chain_b in d:
        chain_a = normalize(unicode(chain_a))
        chain_b = normalize(unicode(chain_b))
        filter_a = (sc.chain == chain_a) if chain_a != 'none' else (sc.chain.isnull())
        filter_b = (sc.chain_bkg == chain_b) if chain_b != 'none' else (sc.chain_bkg.isnull())
        filter_b = filter_b & ~(sc.name_bkg.str.contains(chain_a))
        filter_a = filter_a & ~(sc.name.str.contains(chain_b))
            
        suspicious = suspicious.append(sc[filter_a & filter_b])

len(suspicious)

8543

In [96]:
suspicious.sample(50)

,amd_id,bkg_id_bkg,name,name_bkg,chain,chain_bkg,lat,lng,lat_bkg,lng_bkg
CIAGS800,700141699.0,727759.0,comfort inn ste gordon hwy,hampton inn augusta gordon highway,comfort inn,hampton hltn,33.4451,-82.07358,33.449329,-82.082344
HSAGPADH,700027551.0,1648591.0,senorio aloha apartamentos,senorio aloha,hrs gds,interhome,36.4955,-4.96204,36.495417,-4.962363
HSZCNABY,700183528.0,452902.0,puck landgasthof,landgasthof puck,hrs gds,tip top,52.9302,10.44366,52.930162,10.443672
AKMSYQCA,700011437.0,406731.0,qandc bar new orleans,ritz carlton new orleans,autograph collection,ritz carlton company llc,29.95,-90.06886,29.954820,-90.071178
YXCLT232,700101467.0,280918.0,americas value inn,america s value inn concord,synxis,americas value inn red lion,35.4411,-80.60843,35.441312,-80.608804
HSHIAAAN,700209763.0,1395797.0,hanting south huaihai road,hanting huai an hainan road,hrs gds,hanting inns,33.5866,119.03027,33.587029,119.030288
HHPHX626,700022697.0,77823.0,hltn scottsdale,aloft scottsdale,hltn corpo,aloft,33.4963,-111.92604,33.501436,-111.921214
HHCLT795,700070452.0,42509.0,hltn charlotte university pl,hltn charlotte university place,hltn corpo,hltn resorts,35.311,-80.74782,35.309098,-80.751187
HSTPEAEI,700152500.0,334491.0,fx taipei nanjing east rd branch,fx taipei nanjing east road branch,hrs gds,furama intl management,25.0521,121.54067,25.052302,121.540976
HXDCUFO8,700036507.0,42444.0,hampton inn decatur al,hampton inn decatur,hampton inns,hampton hltn,34.5753,-87.01677,34.578930,-87.021104


In [59]:
suspicious.to_excel('suspicious_matches_excel.xls', index=False)

# 5. remove suspicious matches and save to public matches file

In [60]:
len(matches)

85805

In [61]:
clean_matches = {k:v for k,v in matches.items() if k not in suspicious.amd_id.values}

len(clean_matches)

77314

In [62]:
# percentage matched
len(clean_matches) * 1.0 / len(amdh)

0.5494602335316149

In [63]:
json_dump_unicode(clean_matches,'matches_excel_clean.json')

# 6. Compute livingscores and append columns to excel file

In [66]:
amdh_full = pd.read_excel('./Amadeus All Properties - FEB 2017 17022017 .xlsx', header=0, index_col='PROPERTY_CODE', skiprows=1)

In [69]:
m = clean_matches

## Load livingscores for all Booking.com properties

In [64]:
with open('livingscore.json') as f:
    lvs = json.load(f)

## Map Livingrooms IDs to Amadeus codes

In [68]:
conn = get_amd_db_conn()
cur = conn.cursor()
query = """SELECT pr.id, pr.position, pr.name, v."vendorId"
           FROM public."Properties" pr
           JOIN public."VendorIds" v
           ON pr.id = v.relation
           WHERE v.vendor = 'amadeus'
        """
cur.execute(query)
lrid_to_amdid = {}
for r in cur.fetchall():
    lrid_to_amdid[r[0]] = r[-1]

amdid_to_lrid = {k:v for v,k in lrid_to_amdid.items()}

In [77]:
m.items()[0]

(u'700151432.0', 350152)

In [74]:
amdid_to_lrid.items()[0]

('YOFAYD02', '0afb7c67-406a-42ff-bfe1-e340df973eed')

In [78]:
amdh_full['DUPE_POOL_ID'].values

array([  5.03919399e+08,   5.03948101e+08,   5.03929932e+08, ...,
         7.00144748e+08,              nan,              nan])

In [ ]:
matches_ls = {}
for did, bkgid in m.items():
    matches_ls[did] = lvs.get(str(bkgid))
matches_ls = {k:v for k,v in matches_ls.items() if v}

def max_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return max(ls)
    else:
        return None

def min_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return min(ls)
    else:
        return None

def avg_ls(did):
    ls = matches_ls.get(str(did))
    if ls:
        return np.mean(ls)
    else:
        return None

amdh_full['LIVINGSCORE_MAX'] = amdh_full['DUPE_POOL_ID'].apply(max_ls)
amdh_full['LIVINGSCORE_MIN'] = amdh_full['DUPE_POOL_ID'].apply(min_ls)
amdh_full['LIVINGSCORE_AVG'] = amdh_full['DUPE_POOL_ID'].apply(avg_ls)

In [84]:
def did_to_bkg(did):
    return m.get(str(did))

amdh_full['Booking ID'] = amdh_full['DUPE_POOL_ID'].apply(did_to_bkg)

In [86]:
amdh_full.to_excel('Amadeus All Properties - FEB 2017 17022017 with Livingscores .xlsx')